# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{MB}i$ for $i\in\{1,\cdots,n\}$ is the number of missionaries that cross the river
  during the $i^{\textrm{th}}$ crossing.
* $\texttt{CB}i$ for $i\in\{1,\cdots,n\}$ is the number of infidels that cross the river
  during the $i^{\textrm{th}}$ crossing.  

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [1]:
def flatten(LoL):
    return [x for L in LoL for x in L]

In [2]:
flatten([[1,2], [3,4]])

[1, 2, 3, 4]

The function `no_problem(i)` takes an integer `i` of as its input.
It returns a formula that is true if there is no problem on either shore of the river after the $i^{\textrm{th}}$ crossing.  There is no problem if any of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. 
  $\texttt{M}i = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M}i = 3$.
  Then there are no missionaries on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of 
  infidels on that side, i.e. $\texttt{M}i = \texttt{C}i$.  Then the numbers of 
  missionaries and infidels have to match on the eastern shore as well.

In [3]:
def no_problem(i):
    return f'M{i} == 0 or M{i} == 3 or M{i} == C{i}'

In [4]:
no_problem(7)

'M7 == 0 or M7 == 3 or M7 == C7'

The function `boat_ok` is true if the boat is neither empty nor overloaded during the $i^{\textrm{th}}$ crossing, i.e. the number of passengers is either 1 or 2.

In [5]:
def boat_ok(i):
    return f'1 <= MB{i} + CB{i} <= 2'

In [6]:
boat_ok(3)

'1 <= MB3 + CB3 <= 2'

The function `change_ok(i)` returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the $i^{\textrm{th}}$ crossing.  Note that if $i$ is odd, then during the $i^{\textrm{th}}$ crossing the boat travels from the western shore to the eastern shore.  If $i$ is even, the boat travels from the eastern shore to the western shore.

In [7]:
def change_ok(i):
    if i % 2 == 1:
        return { f'M{i+1} == M{i} + MB{i+1}', f'C{i+1} == C{i} + CB{i+1}'}
    else:
        return { f'M{i+1} == M{i} - MB{i+1}', f'C{i+1} == C{i} - CB{i+1}'}

In [8]:
change_ok(0)

{'C1 == C0 - CB1', 'M1 == M0 - MB1'}

In [9]:
change_ok(1)

{'C2 == C1 + CB2', 'M2 == M1 + MB2'}

The function `missionaries_CSP` creates a CSP that tries to solve the problem with `n` crossings.

In [10]:
def missionaries_CSP(n):
        "Returns a CSP encoding the problem."
        Lists        = [[f'MB{i}', f'CB{i}', f'M{i}', f'C{i}'] for i in range(n+1)]
        Variables    = flatten(Lists)
        Variables    = Variables[2:] # remove MB0 and CB0
        Values       = { 0, 1, 2, 3 }
        Constraints  = { 'M0 == 3', 'C0 == 3', f'M{n} == 0', f'C{n} == 0' }
        Constraints |= { no_problem(i) for i in range(0, n+1) }
        Constraints |= { boat_ok(i)    for i in range(1, n+1) }
        for i in range(n):
            Constraints |= change_ok(i)
        return Variables, Values, Constraints

In [11]:
missionaries_CSP(1)

(['M0', 'C0', 'MB1', 'CB1', 'M1', 'C1'],
 {0, 1, 2, 3},
 {'1 <= MB1 + CB1 <= 2',
  'C0 == 3',
  'C1 == 0',
  'C1 == C0 - CB1',
  'M0 == 0 or M0 == 3 or M0 == C0',
  'M0 == 3',
  'M1 == 0',
  'M1 == 0 or M1 == 3 or M1 == C1',
  'M1 == M0 - MB1'})

In [12]:
%run 02-Backtracking-Constraint-Solver.ipynb

The function `find_solution` computes a solution to the problem of saving the infidels.

In [13]:
def find_solution():
    n = 1
    while True:
        print(n)
        CSP = missionaries_CSP(n)
        Solution = solve(CSP)
        if Solution != None:
            return n, Solution
        n += 2

On my desktop computer (2017 iMac with 3.4 GHz Quad-Core Intel i5) it takes about 2 seconds to solve the problem. 

In [14]:
%%time
n, Solution = find_solution()
n, Solution

1
3
5
7
9
11
CPU times: user 2.65 s, sys: 2.93 ms, total: 2.66 s
Wall time: 1.16 s


(11,
 {'M0': 3,
  'C0': 3,
  'MB1': 0,
  'CB1': 2,
  'M1': 3,
  'C1': 1,
  'MB2': 0,
  'CB2': 1,
  'M2': 3,
  'C2': 2,
  'MB3': 0,
  'CB3': 2,
  'M3': 3,
  'C3': 0,
  'MB4': 0,
  'CB4': 1,
  'M4': 3,
  'C4': 1,
  'MB5': 2,
  'CB5': 0,
  'M5': 1,
  'C5': 1,
  'MB6': 1,
  'CB6': 1,
  'M6': 2,
  'C6': 2,
  'MB7': 2,
  'CB7': 0,
  'M7': 0,
  'C7': 2,
  'MB8': 0,
  'CB8': 1,
  'M8': 0,
  'C8': 3,
  'MB9': 0,
  'CB9': 2,
  'M9': 0,
  'C9': 1,
  'MB10': 0,
  'CB10': 1,
  'M10': 0,
  'C10': 2,
  'MB11': 0,
  'CB11': 2,
  'M11': 0,
  'C11': 0})

In [15]:
def show_solution(Solution, n):
    for i in range(n+1):
        M  = Solution[f'M{i}']
        C  = Solution[f'C{i}']
        if i + 1 < n:
            MB = Solution[f'MB{i+1}']
            CB = Solution[f'CB{i+1}']
        print('😇' * M + '🥷' * C + ' ' * 28 + '😇' * (3 - M) + '🥷' * (3 - C))
        if i % 2 == 0:
            print(' ' * 12 + '>>> ' + '😇'*MB + '🥷'*CB + ' >>>')
        elif i % 2 == 1 and i + 1 < n:
            print(' ' * 12 + '<<< ' + '😇'*MB + '🥷'*CB + ' <<<')

In [16]:
show_solution(Solution, n)

😇😇😇🥷🥷🥷                            
            >>> 🥷🥷 >>>
😇😇😇🥷                            🥷🥷
            <<< 🥷 <<<
😇😇😇🥷🥷                            🥷
            >>> 🥷🥷 >>>
😇😇😇                            🥷🥷🥷
            <<< 🥷 <<<
😇😇😇🥷                            🥷🥷
            >>> 😇😇 >>>
😇🥷                            😇😇🥷🥷
            <<< 😇🥷 <<<
😇😇🥷🥷                            😇🥷
            >>> 😇😇 >>>
🥷🥷                            😇😇😇🥷
            <<< 🥷 <<<
🥷🥷🥷                            😇😇😇
            >>> 🥷🥷 >>>
🥷                            😇😇😇🥷🥷
            <<< 🥷 <<<
🥷🥷                            😇😇😇🥷
            >>> 🥷 >>>
                            😇😇😇🥷🥷🥷
